In [ ]:
import pandas as pd

# Reading csv as dataframe
g20_index_data = pd.read_csv("UNION.csv")
g20_covid = pd.read_csv("g20_covid.csv")

# g20_index_data.head()
# g20_covid.head()

: 

In [ ]:
g20_covid.info()

In [ ]:
# Cleaning Dataset
g20_index_data['USD'] = g20_index_data['USD'].replace(0, None)
g20_index_data.ffill(inplace=True)

# As we only have COVID data from 